# Notes

In [ ]:
#  You gotta run the following sections in order for it to work:
#  Libraries Imports --> Classes --> Main Declarations --> Initialization --> Searching

# Main entry point has the urls that have been used as the main initial seeds of data 
# for the crawler and for the whole engine

# Libraries Imports

In [2]:
import numpy as np # for mathmatical operations
import nltk # for language processing purposes
import requests # for doing http requests
import re # for regular expressions usage
from bs4 import BeautifulSoup # For extracting infomration from raw html
from nltk.tokenize import sent_tokenize, word_tokenize # for tokenizing sentences and words
#from nltk.stem import ISRIStemmer as ArStemmer # Arabic stemmer
from nltk.stem import PorterStemmer as EnStemmer # English stemmer
from nltk.corpus import stopwords # for latest conventional stop words
from nltk.corpus import wordnet as wn # for getting synonyms of a word and finding similarities
import time # for time measurements
import sklearn # for data splitting and model training
import pandas as pd # for reading 
import csv # for writing
from sklearn.model_selection import train_test_split # for splitting data into test and training portions
from sklearn.linear_model import LinearRegression # for creating a learn regresssion model
from sklearn.tree import DecisionTreeRegressor # for creating a desicion tree regressor model
import hashlib # for getting the hash of a string
from joblib import load, dump # for loading and saving trained models

# Main Declarations

In [3]:
Documents = list()
Index = dict()
ModelInfo = list()
englishStopwords = Helper().AddMoreStopWords(stopwords.words("english"))
enStemmer = EnStemmer()

documentsPath = "foodSearchEngineDocuments.txt"
indexPath = "foodSearchEngineIndex.txt"
mlModelPath = "foodSearchEngineMlModel.txt"
traningModelDataPath = "foodSearchEngineTrainingData.csv"
trainedModelPath = "rankerModel.joblib"

# Initialization

In [4]:
Documents = eval(FileManager().Load(documentsPath))
Index = eval(FileManager().Load(indexPath))
ModelInfo = eval(FileManager().Load(mlModelPath))
Model = FileManager().LoadRankerModel(trainedModelPath)

# Classes

In [21]:
# Helper class for some general usage functions
class Helper:
    def AddMoreStopWords(self, stopwords):
        stopwords.append(".")
        stopwords.append("!")
        stopwords.append("%")
        stopwords.append("#")
        stopwords.append("$")
        stopwords.append("^")
        stopwords.append("*")
        stopwords.append("&")
        stopwords.append("?")
        stopwords.append(",")
        stopwords.append(":")
        stopwords.append(";")
        stopwords.append("(")
        stopwords.append(")")
        stopwords.append("[")
        stopwords.append("]")
        
        for i in range(10):
            stopwords.append(str(i))
        return stopwords
    
    def CreateDataFrame(self, data, forPrediction = True):
        # define the columns
        if (forPrediction == False):
            columns = ['query','doc_tf','url_tf','doc_idf','url_idf','doc_tf-idf','url_tf-idf','url_length','doc_length', 'page_rank', 'outgoing_links']
        else:
            columns = ['query','doc_tf','url_tf','doc_idf','url_idf','doc_tf-idf','url_tf-idf','url_length','doc_length', 'outgoing_links']
        
        # Read the data using the columns as a dataframe
        return pd.DataFrame([[data[i][c] for c in columns] for i in range(len(data))], columns= columns)
        
        
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #

# Cralwer class for retrieving urls of a specific website 
class Crawler:
    
    def __init__(self):
        self.ToVisit = list()
        self.Visited = set()
        
# --------------------------------------------------------- #

    # Retrieves the relevent text from a raw html string
    def TextFromHtml(self, html):
        soup = BeautifulSoup(html)

        # get ride of all script and style elements
        for script in soup(["script", "style"]):
            script.extract()    # rip it out

        # get text
        text = soup.get_text()

        # break into lines and remove leading and trailing space on each
        lines = (line.strip() for line in text.splitlines())
        # break multi-headlines into a line each
        chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
        # drop blank lines
        text = '\n'.join(chunk for chunk in chunks if chunk)

        return text
        
# --------------------------------------------------------- #

    def Fetch(self, url):
        print ('Fetching... ', url)
        try:
            return requests.get(url).content
        except:
            return ""
    
# --------------------------------------------------------- #

    def GetCurrentUrlToFetch(self):
        res = self.ToVisit.pop()
        
        while res in self.Visited:
            if (len(self.ToVisit) < 1 and res in self.Visited):
                return ""
            res = self.ToVisit.pop()
            
        return res
    
# --------------------------------------------------------- #

    def GetUrlLinks(self, pageContent):
        urls = re.findall('<a href="([^"]+)">', str(pageContent))
        pattern = re.compile('https?')
        urlsToReturn = set()
        
        for url in urls:
            if pattern.match(url):
                self.ToVisit.append(url)
                urlsToReturn.add(url)
        return urlsToReturn

# --------------------------------------------------------- #

    def Crawl(self, urls, depth = 30):
        counter = 1
        urlsLength = len(urls)
        
        for url in urls:
            self.ToVisit.append(url)
            print("Progress: ", (counter) * 100 / urlsLength, " %")
            counter = counter + 1
            while len(self.Visited) < depth and len(self.ToVisit) > 0:
                currentUrl = self.GetCurrentUrlToFetch()
                
                if (currentUrl == ""):
                    continue
                
                content = self.Fetch(currentUrl)
                
                if content == "":
                    print(f"Failed to fetch url: {currentUrl}")
                    continue
                
                self.Visited.add(currentUrl)
                urls = self.GetUrlLinks(content)
                text = self.TextFromHtml(content)
                
                Documents.append({
                    "url": currentUrl,
                    "content": text,
                    "urls": urls
                })
                
                print("Finished fetching url: ", currentUrl)
                
            self.ToVisit.clear()
            self.Visited.clear()
        
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #

# Ranker class for ranking documents
class Ranker:
    
    def __init__(self, documents):
        self.totalNodes = len(documents)
        self.graph = self.GraphFromDocuments(documents)
        self.pageRank = np.zeros(self.totalNodes)

# --------------------------------------------------------- #

    def PrintRanks(self):
        for k in range(self.totalNodes):
            print("Page rank of ", k, " is :\t", self.pageRank[k])

# --------------------------------------------------------- #

    # Retrieves the outgoing links for a document in a graph
    def OutgoingLinks(self, nodeNumber):
        return np.sum(self.graph[nodeNumber][:])
    
# --------------------------------------------------------- #
    
    def GraphFromDocuments(self, documents):
        documentsLength = len(documents)
        
        linksGraph = np.zeros((documentsLength, documentsLength))
        
        for i in range(documentsLength):
            for j in range(documentsLength):
                # if we are not on the same document
                if (i != j):
                    linksGraph[i][j] = documents[j]["url"] in documents[i]["urls"]
                    
        return linksGraph
    
# --------------------------------------------------------- #
    
    def CalculateRanks(self):
        
        outgoingLinks = 0
        dampingFactor = 0.85
        tempPageRank = np.zeros(self.totalNodes)
        
        initialPageRank = 1.0 / self.totalNodes
        #print("Total Number of nodes :", self.totalNodes, "\t Initial pageRank of all nodes: ", initialPageRank, "\n")
        
        # Initialization phase
        for k in range(self.totalNodes):
            self.pageRank[k] = initialPageRank
            
        #print("\n Initial PageRank Values, Step 1\n")
        
        #self.PrintRanks()
        
        iterations = 10
        
        for i in range(iterations):
            
            # Store the PageRank for All Nodes in Temporary Array
            for k in range(self.totalNodes):
                tempPageRank[k] = self.pageRank[k]
                self.pageRank[k] = 0
                
            for internalNodeNumber in range(self.totalNodes):
                for externalNodeNumber in range(self.totalNodes):
                    if (self.graph[externalNodeNumber][internalNodeNumber] == 1):
                        outgoingLinks = self.OutgoingLinks(externalNodeNumber)

                        # Calculate PageRank
                        self.pageRank[internalNodeNumber] += tempPageRank[externalNodeNumber] * (1.0 / outgoingLinks)
                        
            #print("After step ", i, " \n")
            #self.PrintRanks()
        return self.pageRank
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #            

class FileManager:
    
    def Load(self, filename):
        content = ""
        with open(filename, 'r+', encoding= "utf-8") as file:
            content = file.read()
        return content
    
# --------------------------------------------------------- #
    
    def Save(self, filename, content, append = False):
        with open(filename, append == True and 'a' or 'w', encoding= "utf-8") as file:
            file.write(content)
            
# ---------------------------------------------------------- #

    def LoadAsCsv(self, filename):
        return pd.read_csv(filename)

# ---------------------------------------------------------- #

    def SaveRankerModel(self, model, filename):
        dump(model, filename)

# ---------------------------------------------------------- #
    
    def LoadRankerModel(self, filename):
        return load(filename)
    
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #

class IndexBuilder:
    
    # Builds an inverted index using the passed documents
    def BuildIndex(self, documents):
        index = dict()
        documentsLength = len(documents)

        for i in range(documentsLength):
            print("Progress: ", (i + 1) * 100 / documentsLength , " %")
            for sentence in sent_tokenize(documents[i]["content"]):
                for term in [enStemmer.stem(word.lower()) for word in word_tokenize(sentence) if word.lower() not in englishStopwords]:
                    if term in index.keys():
                        documentsList = index.get(term)
                        if i not in documentsList:
                            documentsList.append(i)
                    else:
                        documentsList = []
                        documentsList.append(i)
                    index.update({term: documentsList})
        
        return index
    
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #


class TextAnalyzer:
    
    # Computes the frequency of a word in a content (document content or url)
    def TFInContent(self, word, content, isUrl = False):
        if isUrl == True:
            words = self.WordsFromUrl(content)
        else:
            words = word_tokenize(content)
            
        frequency = [1 for term in words if term == word]
        return len(frequency) / len(words)
    
# --------------------------------------------------------- #

    # Computes the IDF of a word in contents (document's contents or urls)
    def IDFInContents(self, word, contents, areUrls = False):
        if areUrls == True:
            chunks = [self.WordsFromUrl(url) for url in contents]
        else:
            chunks = [sentence for content in contents for sentence in sent_tokenize(content)]

        counter = 1
        
        for chunk in chunks:
            if word in chunk:
                counter += 1

        return np.log(len(chunks) / counter)
        
# --------------------------------------------------------- #

    def WordsFromUrl(self, url):
        words = url.split("//")[1].split("/")
        wordsInDomain = words[0].split(".")
        words.append(wordsInDomain["www" in wordsInDomain and 1 or 0])
        del words[0]
        
        return words

# --------------------------------------------------------- #
    
    def ExtractInfoForModel(self, query, documentsResults, forPrediction = True):
        info = list()
        
        if len(documentsResults) < 1:
            return info
        
        # Get all urls returned
        urls = [docResult.document["url"] for docResult in documentsResults]
        
        # Get the content of all the documents returned
        contents = [docResult.document["content"] for docResult in documentsResults]
        
        # Loop over all results and get the info required for the model
        for result in documentsResults:
            doc_tf = self.TFInContent(query, result.document["content"], isUrl = False)
            url_tf = self.TFInContent(query, result.document["url"], isUrl = True)
            doc_idf = self.IDFInContents(query, contents, areUrls = False)
            url_idf = self.IDFInContents(query, urls, areUrls = True)
            url_length = len(result.document["url"])
            doc_length = len(result.document["content"])
            page_rank = result.rank
            outgoing_links = len(result.document["urls"])
            
            if forPrediction == True:
                info.append({
                "query": self.Hash(query),
                "doc_tf": doc_tf,
                "url_tf": url_tf,
                "doc_idf": doc_idf,
                "url_idf": url_idf,
                "doc_tf-idf": doc_tf * doc_idf,
                "url_tf-idf": url_tf * url_idf,
                "url_length": url_length,
                "doc_length": doc_length,
                "outgoing_links": outgoing_links
                })
                
            else:
                # Add info extracted
                ModelInfo.append({
                    "query": query,
                    "doc_tf": doc_tf,
                    "url_tf": url_tf,
                    "doc_idf": doc_idf,
                    "url_idf": url_idf,
                    "doc_tf-idf": doc_tf * doc_idf,
                    "url_tf-idf": url_tf * url_idf,
                    "url_length": url_length,
                    "doc_length": doc_length,
                    "page_rank": page_rank,
                    "outgoing_links": outgoing_links
                })
        
        if forPrediction == True:
            return Helper().CreateDataFrame(info, forPrediction = True)
        
        print("Done extracting info of query: ", query)
        
# --------------------------------------------------------- #

    def Hash(self, text):
        return int(hashlib.sha256(text.encode('utf-8')).hexdigest(), 32) % 10**5

# --------------------------------------------------------- #

    # Retrieves the difference between two words as a number of operations (edits, inserts, updates)
    # Minimum Edit Distance
    def MinEditDistance(self, word1, word2):
        res = np.zeros((len(word1) + 1, len(word2) + 1))

        for i in range(len(word1) + 1):
            res[i,0] = i

        for j in range(len(word2) + 1):
            res[0,j] = j

        for i in range(1, len(word1) + 1):
            for j in range(1, len(word2) + 1):

                c1 = res[i, j - 1] + 1
                c2 = res[i - 1, j] + 1

                if (word1[i - 1] == word2[j - 1]):
                    c3 = res[i - 1, j - 1]
                else:
                    c3 = res[i - 1, j - 1] + 2

                res[i,j] = min(c1, c2, c3)

        return res[len(word1), len(word2)]
    
# --------------------------------------------------------- #
    
    def PredictWordsFromQuery(self, query, limit = 5):
        words = Index.keys()
        similaritiesList = list()
        for word in words:
            similaritiesList.append({
                'word': word,
                'similarity': self.MinEditDistance(query, word)
            })
            
        similaritiesList = sorted(similaritiesList, key= lambda x: x['similarity'])
        
        return [similaritiesList[i]['word'] for i in range(limit)]
        
    
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #

class Searcher:
    
# --------------------------------------------------------- #

    def __init__(self, index, documentsLength):
        self.index = index
        self.documentsLength = documentsLength

    # Does a search
    # Query gonna be:
    #   1- And: query1 and query2 and query3... document that exists in all three query's results
    #   2- Not: query1 not query2 not query3... document that exists only in the first query results
    #   3- Or: query1 or query2 or query3... documents of all queries results combined without redundancy
    def Search(self, query, printResults = False, useRankerModel = True):
        
        # get current time in seconds
        startTime = time.time()
        
        results = list()
        if (" and " in query):
            results = self.AndSearch(query)
        elif (" not " in query):
            results = self.NotSearch(query)
        else:
            # Get the synonyms and do the search including those synonyms
            modifiedQuery = query
            for synonym in self.SynonymsForWordInQuery(query):
                modifiedQuery += f" {synonym}"
                
            results = self.OrSearch(modifiedQuery)
        
        documentsResults = [Documents[index] for index in results]
        
        if (len(documentsResults) < 1):
                print(f"Results: {len(documentsResults)} ({time.time() - startTime:.2f} seconds)")
                return
        
        rankings = list()
        
        if (useRankerModel == True):
            model = FileManager().LoadRankerModel(trainedModelPath)
            predictionInfo = TextAnalyzer().ExtractInfoForModel(query, list([DocumentResult(documentsResults[i]) for i in range(len(documentsResults))]), forPrediction = True)
            rankings = list(model.predict(predictionInfo))
        else:
            rankings = Ranker(documentsResults).CalculateRanks()

        finalResults = [DocumentResult(documentsResults[i], rankings[i]) for i in range(len(rankings))]

        finalResults.sort(key= lambda o: o.rank, reverse = True)
        
        # Send the results for extracting info to train the ranker model
        #TextAnalyzer().ExtractInfoForModel(query, finalResults)

        print(f"Results: {len(finalResults)} ({time.time() - startTime:.2f} seconds)")
        
        # Print the results
        if (printResults):
            for result in finalResults:
                  print(result.document["url"], " With Rank: ", result.rank)
# --------------------------------------------------------- #

    # Retrieves the documents for a spcific term
    def TermSearch(self, term):
        try:
            return self.index[enStemmer.stem(term.lower())]
        except KeyError:
            return list()

# --------------------------------------------------------- #

    # Does a not search for a specific term
    def NotSearch(self, query):
        finalResults = list()
        results = list()
        
        # foreach stemmed lowered non-stopword term from the query splitted by 'not'
        for term in [enStemmer.stem(token.lower()) for token in query.split(' not ') if token.lower() not in englishStopwords]:
            docsResults = self.TermSearch(token)
            if len(docsResults) > 0:
                results.append(docsResults)
        
         # if there are results
        if len(results) > 0 :
            
            # take arrays after the first one.. from 1 to the end
            otherDocsResults = results[1:]

            # foreach document in the first results
            for document in results[0]:

                # indicator to check if the document exists in other documents
                existsInOtherDocs = False

                # check if the current document is in all other documents
                for otherDocs in otherDocsResults:
                    existsInOtherDocs = document in otherDocs
                    # get out if the document doesn't exist in the current set of documents
                    if existsInOtherDocs == True:
                        break

                # if it occured in all other array of docs, add it
                if existsInOtherDocs == False:
                    finalResults.add(document)

        return finalResults

# --------------------------------------------------------- #

    # Retrieves results for an OR search
    def OrSearch(self, text):
        results  = set()
        
        for term in [enStemmer.stem(word.lower()) for word in text.split() if word.lower() not in englishStopwords]:
            for documentIndex in self.TermSearch(term):
                results.add(documentIndex)

        return results

# --------------------------------------------------------- #

    # Retrieves results for an AND search
    def AndSearch(self, text):
        # Inital search results for each token
        results = set()

        # Final results to be returned
        finalResults = set()

        # foreach stemmed lowered non stopword word in the text passed splitted by 'and'
        for token in [enStemmer.stem(word.lower()) for word in text.split(" and ") if word.lower() not in englishStopwords]:
            # Do a normal search for the word
            docsResults = self.TermSearch(token)
            if len(docsResults) > 0:
                results.add(docsResults)

        # if there are results
        if len(results) > 0 :
            # sort the results to work with the smallest array first
            results.sort()

            # take arrays after the first one.. from 1 to the end
            otherDocsResults = results[1:]

            # foreach document in the first results
            for document in results[0]:

                # indicator to check if the document exists in all other documents
                goodToAdd = False

                # check if the current document is in all other documents
                for otherDocs in otherDocsResults:
                    goodToAdd = document in otherDocs
                    # get out if the document doesn't exist in the current set of documents
                    if goodToAdd == False:
                        break;

                # if it occured in all other array of docs, add it
                if goodToAdd == True:
                    finalResults.add(document)

        return finalResults
    
# --------------------------------------------------------- #

    def SynonymsForWordInQuery(self, query):
        syn = set()
        ant = list()
        indexWords = Index.keys()
        for word in query.split():
            for synset in wn.synsets(word):
                for lemma in synset.lemmas():
                    name = lemma.name()
                    if name in indexWords and name != word:
                        syn.add(enStemmer.stem(name))
        return syn
        
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #

class ModelTrainer:
    
    def __init__(self, dataFrame):
        self.dataFrame = dataFrame
    
    def Hash(self, text):
        return int(hashlib.sha256(text.encode('utf-8')).hexdigest(), 32) % 10**5
    
    def TrainModel(self, trianingColumnName):
        X = dataFrame.loc[:, dataFrame.columns != trianingColumnName]
        y = dataFrame[trianingColumnName]
        
        for i in range(len(X)):
            X.iat[i, 0] = self.Hash(X.iat[i, 0])

        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)
        return LinearRegression().fit(X_train, y_train)
        
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
        
class DocumentResult:
    
    def __init__(self, document, rank = 0.0):
        self.document = document
        self.rank = rank
        
    def __lt__(self, other):
        return self.rank < other.rank
        
    def __eq__(self, other):
        return self.rank == other.rank

# Main entry point

In [3]:
urls = [
    "https://www.healthline.com/nutrition/11-most-nutrient-dense-foods-on-the-planet",
    "https://www.myfooddiary.com/foods",
    "https://www.epicurious.com/search",
    "https://www.tasteofhome.com/collection/the-best-soup-recipes",
    "https://www.britannica.com/plant/tomato",
    "https://www.healthline.com/nutrition/foods/tomatoes",
    "https://www.britannica.com/plant/potato",
    "https://www.livescience.com/45838-potato-nutrition.html",
    "https://www.vegetables.co.nz/vegetables-a-z",
    "https://www.choosemyplate.gov/eathealthy/fruits",
    "https://www.fruitatwork.com.au/how-it-works/fruit-information",
    "https://www.healthyeating.org/healthy-eating/all-star-foods/fruits",
    "https://www.food.com",
    "https://foodnetwork.co.uk/?utm_source=foodnetwork.com&utm_medium=domestic",
    "https://www.nytimes.com/section/food",
    "https://www.eatthis.com/classic-comfort-foods-no-longer-eaten",
    "https://www.eatright.org/food",
    "https://www.takingcharge.csh.umn.edu/how-does-food-impact-health",
    "https://www.nutrition.gov/topics/whats-food",
    "http://foodhealthlegal.com/?p=140",
    "https://www.disabled-world.com/fitness",
    "https://www.nhs.uk/live-well",
    "https://www.rbkc.gov.uk/business-and-enterprise/regulation/food-safety/food-information-regulations",
    "https://informationisbeautiful.net/subject/food/",
    "https://www.fsai.ie/",
    "https://www.cdc.gov/healthyweight/",
    "https://health.hawaii.gov/san/food-information/",
    "https://www.sfa.gov.sg/food-information"
]

#Crawler().Crawl(urls, 10)

# Searching

In [10]:
# User enters a query and the system gets all words close to the word he typed
TextAnalyzer().PredictWordsFromQuery('foo')

['food', 'foot', 'floor', 'f', 'foodi']

In [22]:
# User chooses from the words infront of him and does a search
Searcher(Index, len(Documents)).Search("food", printResults = True, useRankerModel = True)

Results: 71 (15.49 seconds)
http://ec.europa.eu/food/safety/novel_food/catalogue/search/public/index.cfm  With Rank:  0.01516819216241292
https://www.food.com/ideas/top-grilling-recipes-6974#c-810376  With Rank:  0.014554780403812449
https://health.hawaii.gov/san/food-information/  With Rank:  0.014545448901118864
https://www.eatthis.com/advertising-opportunities/  With Rank:  0.014114827229359016
https://www.fruitatwork.com.au/how-it-works/fruit-delivery-sunshine-coast  With Rank:  0.013755141818964891
https://www.eatthis.com/partner-sites/  With Rank:  0.013671311667667975
https://www.rbkc.gov.uk/business-and-enterprise/regulation/food-safety/food-information-regulations  With Rank:  0.01354402992687983
https://www.eatthis.com/classic-comfort-foods-no-longer-eaten  With Rank:  0.013332348295211115
https://www.fruitatwork.com.au/how-it-works/fruit-delivery-toowoomba  With Rank:  0.013235977479194708
https://www.fruitatwork.com.au/how-it-works/fruit-delivery-penrith  With Rank:  0.0130